# Join the extractions DataFrames

In [4]:
from pathlib import Path
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
import warnings

warnings.simplefilter('ignore')

required_columns = ["sample_id", "ref_id", "ewoc_code", "valid_time", "extract", "h3_l3_cell", "geometry"]
dataset_folder = Path("/vitodata/worldcereal_data/EXTRACTIONS/all_datasets/")
target_folder = dataset_folder / "grouped_datasets"
all_datasets = list(dataset_folder.glob("*.geoparquet"))

all_datasets = sorted(all_datasets, key=lambda x: x.stem)

maximum_size_mb = 500 # 1.5 GB per dataset

dataframe_groups = []
group_memory_usage_mb = 0
group = []
group_idx = 0

for f_path in tqdm(all_datasets):
    # Make sure all the datasets are in the same CRS
    df = gpd.read_parquet(f_path).to_crs(epsg=4326)
    # Only keep the relevant columns necessary for extractions
    df = df[required_columns]
    # Remove all the datasets with land-cover and not crop-type
    df = df[(df.ewoc_code > 1100000000) & (df.ewoc_code < 1200000000)]

    group.append(df)
    group_memory_usage_mb += df.memory_usage(deep=True).sum() / 1e6
    print('After adding dataset: {} group size is: {:.1f} MB'.format(f_path.stem, group_memory_usage_mb))

    if (group_memory_usage_mb > maximum_size_mb) or (f_path == all_datasets[-1]):
        print('Writing group with size: ', group_memory_usage_mb, 'MB to disk with ', len(group), 'datasets')
        group = pd.concat(group, axis=0)
        group.to_parquet(
            target_folder / f"group_{group_idx}.geoparquet"
        )
        # Clear up-memory and reset variables
        del group
        group = []
        group_memory_usage_mb = 0
        group_idx += 1

  0%|          | 0/132 [00:00<?, ?it/s]

  2%|▏         | 3/132 [00:00<00:10, 12.87it/s]

After adding dataset: 00_2017_AF_One-Acre-Fund-MEL_POINT_110 group size is: 0.3 MB
After adding dataset: 00_2019_KEN_NHI-CROP-HARVEST_POINT_100 group size is: 0.3 MB
After adding dataset: 01_2017_KEN_IIASA_POINT_100 group size is: 0.3 MB
After adding dataset: 01_2018_AF_One-Acre-Fund-MEL_POINT_110 group size is: 0.9 MB


  6%|▌         | 8/132 [00:00<00:07, 17.15it/s]

After adding dataset: 01_2019_KEN_FAO-WAPOR-1_POLY_111 group size is: 1.0 MB
After adding dataset: 01_2019_KEN_RadiantEarth-01_POLY_101 group size is: 1.0 MB
After adding dataset: 01_2019_KEN_RadiantEarth-01_POLY_111 group size is: 1.0 MB
After adding dataset: 01_2021_RWA_FAO-WAPOR-Muvu_POLY_111 group size is: 1.2 MB
After adding dataset: 02_2017_KEN_IIASA_POLY_100 group size is: 1.3 MB


...

 98%|█████████▊| 130/132 [10:40<00:58, 29.35s/it]

After adding dataset: 59_2021_EST_Eurocrops_POLY_110 group size is: 30.7 MB


 99%|█████████▉| 131/132 [11:09<00:29, 29.07s/it]

After adding dataset: 62_2020_FIN_LPIS_POLY_110 group size is: 197.6 MB
After adding dataset: 62_2021_FIN_LPIS_POLY_110 group size is: 362.7 MB
Writing group with size:  362.720546 MB to disk with  3 datasets


100%|██████████| 132/132 [12:09<00:00,  5.52s/it]


In [2]:
group = pd.concat(group, axis=0)
group.to_parquet(
    target_folder / f"grouped.geoparquet"
)

In [3]:
group

,sample_id,ref_id,ewoc_code,valid_time,extract,h3_l3_cell,geometry
3,2021_EST_Eurocrops_POLY_110_19996106,2021_EST_Eurocrops_POLY_110,1101010011,2021-05-02,0,831136fffffffff,"POLYGON ((27.42837 58.11975, 27.42839 58.11972..."
4,2021_EST_Eurocrops_POLY_110_19990620,2021_EST_Eurocrops_POLY_110,1111020030,2021-07-05,0,831136fffffffff,"POLYGON ((26.66816 57.82049, 26.66815 57.82050..."
7,2021_EST_Eurocrops_POLY_110_19990637,2021_EST_Eurocrops_POLY_110,1111020030,2021-07-05,0,831136fffffffff,"POLYGON ((26.30346 58.16094, 26.30393 58.16091..."
8,2021_EST_Eurocrops_POLY_110_19990646,2021_EST_Eurocrops_POLY_110,1111020030,2021-07-05,0,831136fffffffff,"POLYGON ((26.27513 58.16598, 26.27507 58.16599..."
9,2021_EST_Eurocrops_POLY_110_19990599,2021_EST_Eurocrops_POLY_110,1111020030,2021-07-05,0,831136fffffffff,"POLYGON ((26.50085 58.16810, 26.50086 58.16809..."
...,...,...,...,...,...,...,...
1078439,2021_FIN_LPIS_POLY_110_1086350,2021_FIN_LPIS_POLY_110,1101020003,2021-06-01,0,831122fffffffff,"POLYGON ((28.05545 60.75891, 28.05598 60.75839..."
1078442,2021_FIN_LPIS_POLY_110_1086353,2021_FIN_LPIS_POLY_110,1101020003,2021-06-01,0,831122fffffffff,"POLYGON ((27.97927 60.84420, 27.97937 60.84399..."
1078449,2021_FIN_LPIS_POLY_110_1086360,2021_FIN_LPIS_POLY_110,1111010000,2021-06-01,0,831122fffffffff,"POLYGON ((28.04229 60.85389, 28.04227 60.85386..."
1078452,2021_FIN_LPIS_POLY_110_1086363,2021_FIN_LPIS_POLY_110,1101040000,2021-06-01,0,831122fffffffff,"POLYGON ((27.94013 60.76693, 27.94053 60.76623..."


In [1]:
from tqdm import tqdm
import geopandas as gpd
import pandas as pd
from pathlib import Path

dataset_folder = Path("/vitodata/worldcereal_data/EXTRACTIONS/all_datasets/grouped_datasets/")

datasets = []
for dataset in tqdm(list(dataset_folder.iterdir())):
    print(dataset.stem)
    gdf = gpd.read_parquet(dataset)
    print('dataset read')
    gdf = gdf[gdf.extract == 1]
    datasets.append(gdf)
    del gdf

datasets = pd.concat(datasets, axis=0)
datasets.to_parquet('/data/users/Public/couchard/wc_phase2_samplings.parquet')

  0%|          | 0/7 [00:00<?, ?it/s]

group_0
dataset read


 14%|█▍        | 1/7 [00:10<01:04, 10.77s/it]

group_1
dataset read


 29%|██▊       | 2/7 [00:27<01:12, 14.56s/it]

...

 71%|███████▏  | 5/7 [01:16<00:32, 16.31s/it]

group_5
dataset read


 86%|████████▌ | 6/7 [01:45<00:20, 20.40s/it]

group_6
dataset read


100%|██████████| 7/7 [02:02<00:00, 17.46s/it]
